DenseNet model with weights available for benign/malignant classification

In [47]:
##### Predicting the class labels #########

import os
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import pandas as pd
import sys
from breast_cancer_detection.model import model 


In [46]:
main_dir = "/media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project"
pretrained_model_dir = f"{main_dir}/WSI_Breast_Cancer_Classification/breast_cancer_detection"
model_path_dir = f'{pretrained_model_dir}/saved/'
model_config = f"{model_path_dir}/config.json"
model_path = f"{model_path_dir}/model_best.pth"

# Ensure the directory containing custom modules is in the path
sys.path.append(f'{main_dir}/WSI_Breast_Cancer_Classification/breast_cancer_detection')

class ImageFolderWithPaths(ImageFolder):

    def __getitem__(self, index):
  
        img, label = super(ImageFolderWithPaths, self).__getitem__(index)
        
        path = self.imgs[index][0]
        
        return (img, label ,path)



def load_pretrained_model(model_path):
    # Load the entire checkpoint
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
    
    # Assuming you're using a ResNet and it's defined correctly
    myModel = model.densenet121()  # Ensure this uses the correct block and layer definitions as used during training
    
    # Load state dictionary
    myModel.load_state_dict(checkpoint['state_dict'])

    myModel.eval()
    return myModel


# Assuming the model expects images to be 224x224 pixels
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to fit the model input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])


def predict_and_save(model, image_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    
    with torch.no_grad():
        for image, label, path in image_loader:  # Assuming image_loader yields both images and their paths
            # image = image.to(device)
            output = model(image)
            _, predicted = torch.max(output, 1)
            head, tail = os.path.split(path[0])
            predictions.append((os.path.splitext(tail)[0], predicted.item()))  # Collect filename and predicted label

    return predictions


def save_predictions_to_csv(predictions, output_file='Labels/Predicted_labels_benign_malignant.csv'):
    # Convert predictions list to DataFrame
    df = pd.DataFrame(predictions, columns=['Filename', 'Label'])
    # Save to CSV
    df.to_csv(output_file, index=False)


# Load the model
myModel = load_pretrained_model(model_path)

# Directory containing images
image_directory =  f"{main_dir}/Tiles_Data" 
image_files = [os.path.join(image_directory, img) for img in os.listdir(image_directory) if img.endswith('.png')]

# Setup DataLoader
dataset = ImageFolderWithPaths(root=image_directory, transform=transform)
image_loader = DataLoader(dataset, batch_size=1, shuffle=False)  # Batch size set to 1 to handle images one by one


# Get predictions
predictions = predict_and_save(myModel, image_loader)

# Save predictions to CSV
save_predictions_to_csv(predictions)



In [48]:
myModel = torch.load(model_path, map_location=torch.device('cpu'))
myModel

In [ ]:
class ImageFolderWithPaths(ImageFolder):

    def __getitem__(self, index):
  
        img, label = super(ImageFolderWithPaths, self).__getitem__(index)
        
        path = self.imgs[index][0]
        
        return (img, label ,path)

dataset = ImageFolderWithPaths(image_directory,transform=transform) # add transformation directly

dataloader = DataLoader(dataset)


for image, labels, path in dataloader:
    print(os.path.split(path))

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from breast_cancer_detection.model import model 
import torch.optim as optim


def load_pretrained_model(model_path, num_classes=2):
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
    
    # Load your custom model
    myModel = model.densenet121()  # Replace this with the actual function if different
    
    # Assuming the classifier might need to be adjusted
    num_ftrs = myModel.classifier.in_features
    myModel.classifier = torch.nn.Linear(num_ftrs, num_classes)

    # Load state dictionary
    myModel.load_state_dict(checkpoint['state_dict'])

    myModel.eval()  # Set the model to evaluation mode
    return myModel


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25, device='cpu'):
    best_acc = 0.0  # Initialize the best accuracy
    best_model_path = 'best_model.pth'  # Define path to save the best model

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        # Training phase
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}, Acc: {val_acc:.4f}')

        # Check if this is the best model and save
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f'New best model saved with accuracy: {best_acc:.4f}')

    print('Training complete')
    return model


# Define transformations for the input data
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Setup data directories
data_dir = '/path/to/your/data_directory'  # Adjust as per your directory structure
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

# Create datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# Create DataLoaders
batch_size = 16  # Adjust based on your system capability
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = load_pretrained_model(num_classes=2)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Only optimize the final layer

# Setup the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Call the training function
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device=device)


In [6]:
##### Label prediction example #########

import os
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import sys

main_dir = "/media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project"
pretrained_model_dir = f"{main_dir}/WSI_Breast_Cancer_Classification/breast_cancer_detection"
model_path_dir = f'{pretrained_model_dir}/saved/'
model_config = f"{model_path_dir}/config.json"
model_path = f"{model_path_dir}/model_best.pth"

# Ensure the directory containing custom modules is in the path
sys.path.append(f'{main_dir}/WSI_Breast_Cancer_Classification/breast_cancer_detection')

from breast_cancer_detection.parse_config import ConfigParser  


import json
with open(model_config) as config_file:
    config_dict = json.load(config_file)

config = ConfigParser(config=config_dict)  

from breast_cancer_detection.model import model 


def load_pretrained_model(model_path):
    # Load the entire checkpoint
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
    
    # Assuming you're using a ResNet and it's defined correctly
    myModel = model.densenet121()  # Ensure this uses the correct block and layer definitions as used during training
    
    # Load state dictionary
    myModel.load_state_dict(checkpoint['state_dict'])

    # myModel = model.ResNet(block=model.BasicBlock, num_blocks=[3, 4, 6, 3], num_classes=2)
    # myModel.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    myModel.eval()
    return myModel

# Define a function to preprocess images
def preprocess_image(image_path):
    # Assuming the model expects images to be 224x224 pixels
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to fit the model input
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Function to predict labels
def predict(model, image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()


tiles_dir =  f"{main_dir}/test_tiles" 
save_model_path = f"{main_dir}/WSI_Breast_Cancer_Classification/Model_Weights"


# Load the model
myModel = load_pretrained_model(model_path)

# Directory containing images
image_directory =  f"{main_dir}/test_tiles/n_14_test" 
image_files = [os.path.join(image_directory, img) for img in os.listdir(image_directory) if img.endswith('.png')]

# Predict labels for each image
predictions = {img: predict(myModel, img) for img in image_files}

# Print the predictions
for img_path, label in predictions.items():
    print(f'Image: {img_path}, Predicted Label: {"Benign" if label == 0 else "Malignant"}')


Image: /media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project/test_tiles/n_14_test/SUB_n_14_tile_(14_41).png, Predicted Label: Malignant
Image: /media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project/test_tiles/n_14_test/SUB_n_14_tile_(102_29).png, Predicted Label: Benign
Image: /media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project/test_tiles/n_14_test/SUB_n_14_tile_(102_31).png, Predicted Label: Benign
Image: /media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project/test_tiles/n_14_test/SUB_n_14_tile_(103_28).png, Predicted Label: Malignant
Image: /media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project/test_tiles/n_14_test/SUB_n_14_tile_(103_29).png, Predicted Label: Benign
Image: /media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project/test_tiles/n_14_test/SUB_n_14_tile_(103_30).png, Predicted Label: Malignant
Image: /

In [ ]:
##### does not have model weights available #########

import torch
from torchvision import transforms
from PIL import Image
from TransMIL import * 
from TransMIL.models import TransMIL
from TransMIL import MyOptimizer, MyLoss
from TransMIL.MyOptimizer import *


# Assume you have a function to load the model and it's been trained or a pre-trained model is loaded
model = TransMIL()
model.load_state_dict(torch.load('trans_mil_model.pth'))
model.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize if necessary
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def classify_patch(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()


# create file directory structure and train-test split (one time run when the file structure is not created)
main_dir = "/media/ist/Drive2/MANSOOR/Neuroimaging-Project/Breast_Cancer_Classification_Project"

tiles_dir =  f"{main_dir}/test_tiles" 
save_model_path = f"{main_dir}/WSI_Breast_Cancer_Classification/Model_Weights"

# Example usage
image_path = f'{tiles_dir}/n_14_test/SUB_n_14_tile_(6_38).png'
prediction = classify_patch(image_path)
print("Predicted class:", "Benign" if prediction == 0 else "Malignant")
